In [1]:
import collections
import cv2
import itertools as it
import numpy as np
import operator as op
import os
import pickle
import random

from utilities import *

In [2]:
top, left, bottom, right = 609, 381, 625, 501
width, height = right - left, bottom - top

file_titles = [
    'ancient_apparition',
    'anti-mage',
    'broodmother',
    'centaur_warrunner',
    'clinkz',
    'io',
    'juggernaut',
    'keeper_of_the_light',
    "nature's_prophet",
    'nyx_assassin',
    'outworld_devourer',
    'queen_of_pain',
]
s = ''.join(c for c, _ in it.groupby(sorted(it.chain.from_iterable(file_titles))))
print(len(file_titles), s, len(s))

12 '-_abcdefghijklmnopqrstuvwxyz 29


In [3]:
def fn(file_title):
    with open(fr"D:\Dota 2\Heroes\Pickles\{file_title}.pickle", 'rb') as fin:
        return tuple(pickle_iter(fin))
data = {s: fn(s) for s in file_titles}
#show_and_wait(np.vstack(list(data.items())[0][1][:33]))
cv2.destroyAllWindows()
image = list(data.items())[0][1][0]
print([len(v) for _, v in data.items()])
len(data), image.shape, image.dtype, image.min(), image.max()

[23400, 26970, 27030, 25110, 27030, 28920, 26730, 13020, 25500, 27000, 26430, 25830]


(12, (16, 120, 3), dtype('uint8'), 0, 255)

In [4]:
# Color data adds no value to this methodology.
# Transform all data into greyscale.
data = {k: tuple(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in v) for k, v in data.items()}

In [5]:
range_args = (120, 211, 15)
print(*range(*range_args))
known_thresholds = list(range(*range_args))
known_thresholds[-1] = -1

120 135 150 165 180 195 210


In [6]:
# Based on this experiment, it appears 165 is a good threshold for most of the data but, for "outworld devourer",
# this needs to be no more than 135.  Using an adaptive thresholding mechanism ensures a suitable threshold for all data.
# A given threshold excludes more background pixels in greyscale than in color.
def fn(images):
    computed_thresholds = set()
    def fn(index, threshold):
        image = images[index].copy()
        if threshold > 0:
            image[image < threshold] = 0
        else:
            threshold, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
            computed_thresholds.add(int(threshold))
        return image
    indices = random.sample(range(len(images)), k=99)
    show_and_wait(np.hstack([np.vstack([fn(i, n) for i in indices]) for n in known_thresholds]))
    return min(computed_thresholds), max(computed_thresholds)
g = ((k, fn(v)) for k, v in data.items())
#print(*g, sep='\n')
cv2.destroyAllWindows()

In [7]:
# Determine horizontal limits to extract the text from the background.
cv2.namedWindow('tesst', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
def fn(images):
    while True:
        image = random.choice(images)
        _, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
        if show_and_wait(image) == 'g':
            break
    left_limit, right_limit = 0, image.shape[1]
    while True:
        ch = show_and_wait(np.hstack([np.ones([image.shape[0] - 7, left_limit], dtype=image.dtype) * 128,
                                      image[4:-3, left_limit:right_limit],
                                      np.ones([image.shape[0] - 7, image.shape[1] - right_limit], dtype=image.dtype) * 128]))
        if ch == 's':
            left_limit -= 1
        elif ch == 'f':
            left_limit += 1
        elif ch == 'j':
            right_limit -= 1
        elif ch == 'l':
            right_limit += 1
        elif ch == ' ':
            break
        elif ch == 'q':
            return
        else:
            print(ch, ord(ch))
    return (left_limit, right_limit)
#horizontal_limits = {k: fn(v) for k, v in data.items()}
cv2.destroyAllWindows()

In [8]:
# These are the limits created above.
horizontal_limits = {'ancient_apparition': (14, 117),
          'anti-mage': (33, 98),
          'broodmother': (22, 110),
          'centaur_warrunner': (13, 118),
          'clinkz': (44, 87),
          'io': (58, 74),
          'juggernaut': (27, 104),
          'keeper_of_the_light': (14, 118),
          "nature's_prophet": (12, 120),
          'nyx_assassin': (25, 106),
          'outworld_devourer': (14, 117),
          'queen_of_pain': (21, 110)}

In [9]:
def fn(file_title):
    # Determine possible horizontal extents for the letters in a random sample of the images.
    horizontal_offset = horizontal_limits[file_title][0]
    def fn(image):
        # Apply a threshold to and crop the image.
        _, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
        image = image[4:-3, horizontal_offset:horizontal_limits[file_title][1]]

        # Collect a list of letter extents.
        l = []
        for x in range(image.shape[1] - 1):
            if not image[:, x].any() and image[:, x + 1].any():
                l.append([x + horizontal_offset])
            elif len(l) and image[:, x].any() and not image[:, x + 1].any():
                l[-1].append(x + 1 + horizontal_offset)
        return tuple(map(tuple, l))
    images = random.sample(data[file_title], k=999)
    g = map(fn, images)
    expected_length = len([c for c in file_title if c != '_'])
    s = {t for t in g if len(t) == expected_length and all(len(t) == 2 for t in t)}
    g = map(set, zip(*s))
    return tuple((min(a for a, _ in s), max(b for _, b in s), s) for s in g)
#possible_extents = {k: fn(k) for k in data}

In [10]:
# These are the extents created above.
possible_extents = {
    'ancient_apparition': ((16, 21, {(16, 21)}), (21, 28, {(21, 28), (22, 28), (23, 28)}), (29, 34, {(29, 33), (29, 34)}), (35, 37, {(35, 37)}), (38, 43, {(38, 42), (38, 43), (39, 42)}), (44, 49, {(44, 49)}), (50, 55, {(50, 55)}), (58, 65, {(58, 65), (59, 65)}), (65, 70, {(65, 70), (66, 70)}), (71, 76, {(71, 76), (72, 76)}), (76, 83, {(76, 82), (76, 83)}), (83, 89, {(83, 88), (83, 89)}), (89, 91, {(89, 91)}), (91, 98, {(91, 97), (92, 97), (92, 98)}), (97, 101, {(97, 101), (98, 101), (99, 101)}), (102, 108, {(102, 108)}), (109, 115, {(109, 114), (109, 115)})),
    'anti-mage': ((33, 43, {(33, 42), (34, 42), (35, 41), (35, 42), (35, 43)}), (42, 49, {(42, 49), (43, 49)}), (49, 58, {(49, 56), (49, 57), (50, 56), (50, 57), (50, 58)}), (56, 60, {(56, 60), (57, 60), (58, 60)}), (61, 66, {(61, 66), (62, 65), (62, 66)}), (66, 74, {(66, 74), (67, 74)}), (74, 82, {(74, 82), (75, 82)}), (82, 89, {(82, 89), (83, 89)}), (90, 96, {(90, 95), (91, 95), (91, 96)})),
    'broodmother': ((23, 29, {(23, 29), (24, 29)}), (30, 36, {(30, 36), (31, 36)}), (37, 44, {(37, 44)}), (44, 54, {(44, 53), (45, 53), (45, 54), (46, 53)}), (53, 61, {(53, 60), (54, 60), (54, 61)}), (61, 70, {(61, 70), (62, 69), (62, 70)}), (70, 79, {(70, 78), (71, 78), (71, 79)}), (78, 86, {(78, 85), (79, 85), (79, 86)}), (86, 93, {(86, 93), (87, 93)}), (94, 99, {(94, 99), (95, 99)}), (101, 107, {(101, 107)})),
    'centaur_warrunner': ((14, 20, {(14, 20), (15, 19), (15, 20)}), (21, 25, {(21, 25)}), (26, 32, {(26, 30), (26, 32), (27, 32)}), (30, 38, {(30, 38), (32, 38), (33, 38)}), (38, 43, {(38, 43)}), (44, 49, {(44, 49)}), (50, 55, {(50, 55), (51, 55)}), (59, 67, {(59, 67)}), (67, 72, {(67, 72)}), (73, 79, {(73, 78), (74, 78), (74, 79)}), (79, 84, {(79, 84), (80, 84)}), (85, 92, {(85, 90), (85, 91), (85, 92), (86, 90), (86, 91)}), (92, 97, {(92, 97)}), (97, 104, {(97, 104), (98, 104), (99, 104)}), (105, 109, {(105, 109)}), (110, 116, {(110, 115), (110, 116), (111, 115)})),
    'clinkz': ((44, 52, {(44, 46), (45, 52), (46, 51), (46, 52)}), (46, 59, {(46, 52), (53, 58), (53, 59)}), (53, 62, {(53, 58), (59, 62)}), (59, 70, {(59, 62), (62, 70), (63, 70), (64, 70)}), (62, 78, {(62, 70), (71, 78), (72, 78)}), (71, 85, {(71, 85), (79, 84), (79, 85)})),
    'io': ((58, 63, {(58, 63), (59, 62), (59, 63), (60, 62)}), (62, 72, {(62, 72), (63, 72), (64, 71), (64, 72)})),
    'juggernaut': ((28, 33, {(28, 33), (29, 33)}), (35, 43, {(35, 41), (35, 42), (35, 43)}), (41, 49, {(41, 49), (42, 49), (43, 49)}), (50, 57, {(50, 56), (50, 57), (51, 56)}), (57, 63, {(57, 63), (58, 63)}), (63, 70, {(63, 70), (64, 70), (65, 70)}), (70, 79, {(70, 78), (72, 78), (72, 79)}), (80, 87, {(80, 86), (80, 87)}), (87, 94, {(87, 94), (88, 94)}), (95, 103, {(95, 101), (95, 102), (95, 103)})),
    'keeper_of_the_light': ((14, 21, {(14, 21), (15, 20), (15, 21), (16, 20), (16, 21)}), (22, 27, {(22, 26), (22, 27)}), (27, 32, {(27, 31), (27, 32), (28, 31)}), (33, 37, {(33, 37)}), (38, 43, {(38, 42), (38, 43), (39, 42)}), (44, 49, {(44, 49)}), (53, 59, {(53, 59)}), (60, 64, {(60, 64)}), (65, 73, {(65, 73), (67, 73), (68, 72), (68, 73)}), (73, 79, {(73, 79), (74, 79)}), (80, 85, {(80, 84), (80, 85), (81, 84), (81, 85)}), (89, 93, {(89, 93)}), (94, 97, {(94, 96), (94, 97)}), (96, 103, {(96, 102), (97, 102), (97, 103)}), (104, 109, {(104, 109)}), (110, 117, {(110, 115), (110, 117)})),
    "nature's_prophet": ((14, 20, {(14, 20)}), (21, 28, {(21, 28), (22, 28)}), (28, 34, {(28, 34)}), (35, 43, {(35, 41), (35, 42), (35, 43), (36, 41), (36, 42)}), (43, 49, {(43, 45), (43, 49)}), (46, 56, {(46, 55), (49, 56), (50, 54), (50, 55)}), (56, 59, {(56, 58), (56, 59)}), (58, 64, {(58, 64), (59, 64), (60, 64)}), (69, 76, {(69, 74), (69, 75), (69, 76)}), (75, 89, {(75, 81), (75, 89), (76, 81)}), (81, 91, {(81, 89), (82, 89), (82, 91), (89, 91)}), (89, 97, {(89, 97), (90, 96), (91, 95), (91, 96)}), (97, 105, {(97, 103), (97, 105)}), (105, 110, {(105, 109), (105, 110)}), (110, 118, {(110, 117), (111, 116), (111, 117), (111, 118)})),
    'nyx_assassin': ((27, 41, {(27, 33), (27, 34), (35, 41)}), (34, 48, {(34, 41), (35, 41), (35, 42), (41, 48)}), (41, 52, {(41, 48), (42, 48), (42, 49), (48, 52)}), (50, 60, {(50, 60), (51, 59), (53, 59), (53, 60)}), (60, 65, {(60, 65), (61, 65)}), (65, 72, {(65, 71), (66, 71), (66, 72), (67, 71)}), (72, 79, {(72, 79)}), (79, 85, {(79, 85), (80, 85)}), (86, 91, {(86, 91)}), (91, 96, {(91, 96), (92, 95)}), (95, 103, {(95, 103), (96, 103), (97, 103)})),
    'outworld_devourer': ((16, 22, {(16, 22)}), (22, 29, {(22, 28), (23, 28), (23, 29)}), (29, 34, {(29, 34)}), (34, 42, {(34, 42), (35, 42)}), (43, 50, {(43, 48), (43, 49), (43, 50)}), (50, 55, {(50, 54), (50, 55)}), (54, 65, {(54, 65), (55, 58), (55, 59), (56, 58)}), (60, 74, {(60, 65), (60, 66), (61, 65), (69, 74)}), (66, 79, {(66, 68), (67, 69), (69, 74), (69, 75), (70, 74), (75, 79)}), (69, 85, {(69, 74), (75, 79), (80, 85)}), (75, 92, {(75, 79), (80, 85), (80, 86), (86, 92)}), (80, 98, {(80, 86), (80, 92), (86, 92), (93, 98)}), (86, 104, {(86, 98), (93, 98), (93, 99), (99, 104)}), (98, 109, {(98, 104), (99, 104), (105, 109)}), (105, 114, {(105, 109), (110, 114)}), (110, 116, {(110, 114), (110, 115), (110, 116), (114, 116)})),
    'queen_of_pain': ((23, 38, {(23, 32), (24, 32), (32, 38)}), (32, 40, {(32, 38), (38, 40)}), (38, 45, {(38, 45), (39, 45), (40, 45)}), (45, 53, {(45, 51), (45, 53), (46, 51), (47, 51)}), (53, 72, {(53, 59), (53, 60), (53, 72)}), (62, 78, {(62, 72), (64, 72), (65, 72), (73, 78)}), (73, 82, {(73, 78), (78, 82)}), (82, 88, {(82, 88), (83, 88)}), (88, 96, {(88, 95), (88, 96)}), (96, 99, {(96, 99)}), (100, 107, {(100, 107), (101, 107)})),
}

In [11]:
# Select the horizontal extent of each letter of each Hero.
cv2.namedWindow('tesst', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
def fn(file_title):
    images = data[file_title]
    while True:
        reference_image = random.choice(images)
        _, reference_image = cv2.threshold(reference_image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
        ch = show_and_wait(reference_image)
        if ch == 'g':
            break
        elif ch == 'q':
            return
    selected_extents = []
    for horizontal_extent in possible_extents[file_title]:
        left_offset = right_offset = 0
        while True:
            image = reference_image.copy()
            for selected_extent in selected_extents:
                image[-1, selected_extent[0]:selected_extent[1]] = 128
            image[-2, horizontal_extent[0] + left_offset:horizontal_extent[1] + right_offset] = 255
            ch = show_and_wait(image)
            if ch == 's':
                left_offset -= 1
            elif ch == 'f':
                left_offset += 1
            elif ch == 'j':
                right_offset -= 1
            elif ch == 'l':
                right_offset += 1
            elif ch == ' ':
                break
            elif ch == 'q':
                return
            elif ord(ch) == 8:
                return fn(file_title)
            else:
                print(ch, ord(ch))
        selected_extents.append([horizontal_extent[0] + left_offset, horizontal_extent[1] + right_offset])
    return selected_extents
#selected_extents = {k: fn(k) for k in data}
cv2.destroyAllWindows()

In [12]:
# These are the extents created above.
selected_extents = {
    'ancient_apparition': ((16, 22), (22, 29), (29, 35), (35, 38), (39, 43), (44, 50), (50, 56), (59, 66), (66, 71), (71, 76), (76, 83), (83, 89), (89, 92), (92, 98), (99, 102), (102, 109), (109, 115)),
    'anti-mage': ((35, 43), (43, 50), (50, 57), (58, 61), (62, 66), (67, 75), (75, 83), (83, 90), (91, 96)),
    'broodmother': ((24, 30), (31, 37), (37, 45), (46, 54), (54, 61), (62, 70), (71, 79), (79, 86), (87, 94), (95, 100), (101, 108)),
    'centaur_warrunner': ((15, 21), (21, 26), (26, 33), (33, 39), (39, 44), (44, 50), (50, 56), (59, 68), (68, 73), (74, 79), (80, 85), (86, 92), (92, 98), (99, 105), (105, 110), (110, 116)),
    'clinkz': ((46, 53), (53, 59), (59, 63), (64, 71), (72, 79), (79, 85)),
    'io': ((60, 63), (64, 72)),
    'juggernaut': ((29, 34), (35, 42), (43, 50), (51, 58), (58, 64), (65, 71), (72, 79), (80, 88), (88, 95), (95, 102)),
    'keeper_of_the_light': ((16, 22), (22, 27), (27, 32), (33, 38), (38, 43), (44, 50), (53, 60), (60, 65), (68, 74), (74, 80), (80, 85), (89, 94), (94, 97), (97, 103), (104, 110), (110, 116)),
    "nature's_prophet": ((14, 21), (22, 29), (29, 35), (36, 42), (43, 50), (50, 56), (56, 59), (59, 65), (69, 75), (76, 82), (82, 90), (91, 97), (97, 104), (105, 111), (111, 118)),
    'nyx_assassin': ((27, 34), (35, 42), (42, 49), (53, 60), (60, 66), (66, 72), (72, 80), (80, 86), (86, 92), (92, 96), (97, 104)),
    'outworld_devourer': ((16, 23), (23, 29), (29, 35), (35, 43), (43, 50), (50, 55), (55, 60), (60, 66), (69, 75), (75, 80), (80, 86), (86, 93), (93, 99), (99, 105), (105, 110), (110, 115)),
    'queen_of_pain': ((23, 33), (33, 39), (40, 46), (47, 52), (53, 60), (64, 73), (73, 79), (83, 89), (89, 96), (96, 100), (101, 108)),
}

In [13]:
# Construct and display some random text.
image = next(iter(data.values()))[0]
dtype, height, width = image.dtype, *image.shape
def make_letter():
    file_title, extents = random.choice(tuple(selected_extents.items()))
    left, right = random.choice(extents)
    image = random.choice(data[file_title])
    image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)[1]
    image = image[:, left:right]
    return image
def make_space():
    width = random.randint(0, 1) if random.random() < .8 else random.randint(7, 8)
    image = np.zeros((height, width), dtype=dtype)
    return image
def fn():
    a = (make_letter() for _ in it.repeat(None))
    b = (make_space() for _ in it.repeat(None))
    g = it.chain.from_iterable(zip(a, b))
    n = random.randint(2, 19) * 2 - 1
    g = (image for _, image in zip(range(n), g))
    image = np.hstack(list(g))
    if image.shape[1] < width:
        image = np.hstack([image, np.zeros((height, width - image.shape[1]))])
    else:
        image = image[:, :width]
    return image
show_and_wait(np.vstack([fn() for _ in range(9)]))
cv2.destroyAllWindows()

In [14]:
def fn():
    file_path = r"D:\Dota 2\Heroes\Pickles\letter_image_dict.pickle"
    if os.access(file_path, os.F_OK):
        # Load the previously saved dictionary.
        with open(file_path, 'rb') as fin:
            d = pickle.load(fin)
    else:
        # Create a dictionary of letters to images for each extent extracted from each image in the data dictionary.
        d = collections.defaultdict(list)
        for file_title, images in data.items():
            file_extents = selected_extents[file_title]
            letters = [c for c in file_title if c != '_']
            for (left, right), letter in zip(file_extents, letters):
                for image in images:
                    image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)[1]
                    image = image[:, left:right]
                    d[letter.upper()].append(image)
        # Save that dictionary.
        with open(file_path, 'wb') as fout:
            pickle.dump(letter_images, fout)
    return d
letter_images = fn()

In [15]:
print(len(letter_images), *letter_images)
l = [(k, len(v)) for k, v in letter_images.items()]
print(sum(n for _, n in l))
print(*l, sep='\n')

28 A N C I E T P R O - M G B D H U W L K Z J F ' S Y X V Q
3421770
('A', 306420)
('N', 357420)
('C', 75540)
('I', 218970)
('E', 361950)
('T', 256110)
('P', 136650)
('R', 347940)
('O', 277050)
('-', 26970)
('M', 54000)
('G', 93450)
('B', 27030)
('D', 79890)
('H', 78570)
('U', 207870)
('W', 51540)
('L', 66480)
('K', 40050)
('Z', 27030)
('J', 26730)
('F', 38850)
("'", 25500)
('S', 133500)
('Y', 27000)
('X', 27000)
('V', 26430)
('Q', 25830)


In [16]:
# Display examples of each letter.
def display_examples(d):
    def fn(c):
        width = 11 + max(np.hstack(list(g)).shape[1] for g in grouper(random.sample(d[c], k=2000), 50))
        def fn(g):
            image = np.hstack(list(g))
            image = np.hstack([image, np.zeros((image.shape[0], width - image.shape[1]))])
            return image
        return show_and_wait(np.vstack([fn(g) for g in grouper(random.sample(d[c], k=2000), 50)]))
    g = map(fn, sorted(d))
    _ = list(it.takewhile(lambda c: c != 'q', g))
    cv2.destroyAllWindows()

In [17]:
display_examples(letter_images)

In [18]:
# Create a file that multiple processes can partially load.
file_path = r"D:\Dota 2\Heroes\Pickles\letter_images.pickle"
if not os.access(file_path, os.F_OK):
    with open(file_path, 'wb') as fout:
        # Write a dictionary of letter image counts.
        pickle.dump({k: len(v) for k, v, in letter_images.items()}, fout)
        # Write each letter's images in lexical order.
        for k, v in sorted(letter_images.items()):
            print(k, len(v))
            pickle.dump(tuple(v), fout)

In [19]:
# Test partial loading.
def fn():
    with open(file_path, 'rb') as fin:
        # Read the letter image count dictionary.
        d = pickle.load(fin)
        # Read a fraction of the images for each letter.
        d = {k: random.sample(pickle.load(fin), k=9999) for k in sorted(d)}
    return d
sampled_letter_images = fn()
[(k, len(v)) for k, v in sampled_letter_images.items()]

[("'", 9999),
 ('-', 9999),
 ('A', 9999),
 ('B', 9999),
 ('C', 9999),
 ('D', 9999),
 ('E', 9999),
 ('F', 9999),
 ('G', 9999),
 ('H', 9999),
 ('I', 9999),
 ('J', 9999),
 ('K', 9999),
 ('L', 9999),
 ('M', 9999),
 ('N', 9999),
 ('O', 9999),
 ('P', 9999),
 ('Q', 9999),
 ('R', 9999),
 ('S', 9999),
 ('T', 9999),
 ('U', 9999),
 ('V', 9999),
 ('W', 9999),
 ('X', 9999),
 ('Y', 9999),
 ('Z', 9999)]

In [20]:
# Display examples of each letter.
display_examples(sampled_letter_images)

In [21]:
# Create a dictionary of letters to images for each extent extracted from each image in each video file.
# Distinguish between large and small letter renderings by using an upper-case letter for the large ones and
# a lower-case letter for the small ones.
def fn():
    file_path = r"D:\Dota 2\Heroes\Pickles\sized_letter_images.pickle"
    if os.access(file_path, os.F_OK):
        with open(file_path, 'rb') as fin:
            # Read the dictionary of letter image counts.
            d = pickle.load(fin)
            # Read each letter's images in lexical order.
            for k in sorted(d):
                d[k] = pickle.load(fin)
    else:
        # Create a file that multiple processes can partially load.
        small_letters = {
            'ancient_apparition',
            'centaur_warrunner',
            'keeper_of_the_light',
            'outworld_devourer',
        }
        d = collections.defaultdict(list)
        for file_title, images in data.items():
            file_extents = selected_extents[file_title]
            letters = [c if file_title in small_letters else c.upper() for c in file_title if c != '_']
            for (left, right), letter in zip(file_extents, letters):
                for image in images:
                    image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)[1]
                    image = image[:, left:right]
                    d[letter].append(image)
        with open(file_path, 'wb') as fout:
            # Write a dictionary of letter image counts.
            pickle.dump({k: len(v) for k, v, in d.items()}, fout)
            # Write each letter's images in lexical order.
            for k, v in sorted(d.items()):
                print(k, len(v))
                pickle.dump(tuple(v), fout)
    return d
sized_letter_images = fn()

In [22]:
print(*sorted(sized_letter_images))
def fn(fn):
    all_letters = {fn(chr(i)) for i in range(ord('A'), ord('Z') + 1)}
    given_letters = {c for c in sized_letter_images if c == fn(c)}
    missing_letters = all_letters - given_letters
    print('missing', len(missing_letters), sorted(missing_letters))
fn(str.upper)
fn(str.lower)
[(k, len(v)) for k, v in sorted(sized_letter_images.items())]

' - A B C D E F G H I J K L M N O P Q R S T U X Y Z a c d e f g h i k l n o p r t u v w
missing 2 ['V', 'W']
missing 8 ['b', 'j', 'm', 'q', 's', 'x', 'y', 'z']


[("'", 25500),
 ('-', 26970),
 ('A', 186000),
 ('B', 27030),
 ('C', 27030),
 ('D', 27030),
 ('E', 183390),
 ('F', 25830),
 ('G', 80430),
 ('H', 52530),
 ('I', 135750),
 ('J', 26730),
 ('K', 27030),
 ('L', 27030),
 ('M', 54000),
 ('N', 211890),
 ('O', 161340),
 ('P', 76830),
 ('Q', 25830),
 ('R', 131790),
 ('S', 133500),
 ('T', 131730),
 ('U', 104790),
 ('X', 27000),
 ('Y', 27000),
 ('Z', 27030),
 ('a', 120420),
 ('c', 48510),
 ('d', 52860),
 ('e', 178560),
 ('f', 13020),
 ('g', 13020),
 ('h', 26040),
 ('i', 83220),
 ('k', 13020),
 ('l', 39450),
 ('n', 145530),
 ('o', 115710),
 ('p', 59820),
 ('r', 216150),
 ('t', 124380),
 ('u', 103080),
 ('v', 26430),
 ('w', 51540)]

In [23]:
# Display examples of each letter.
display_examples(sized_letter_images)

The rest of the code is archived and not used.

In [24]:
# Select 1200 of each letter.
# Manually select ranges of images to exclude.
nimages = 1200
ncolumns = 60
window_name = 'tesst'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
def fn(file_title, horizontal_extent):
    images = data[file_title]
    indices = list(range(nimages))
    left, right = horizontal_extent
    def fn():
        g = (cv2.threshold(images[i], 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)[1][:, left:right] for i in indices)
        g = grouper(g, ncolumns)
        return np.vstack([np.hstack(list(g)) for g in g])
    initial_index = []
    height, width = images[0].shape[0], right - left
    def handle_mouse_event(event, x, y, *_):
        if event == cv2.EVENT_LBUTTONDOWN:
            initial_index.clear()
            initial_index.append((y // height) * ncolumns + x // width)
        elif event == cv2.EVENT_RBUTTONDOWN:
            index = (y // height) * ncolumns + x // width
            begin, end = initial_index[0] if initial_index else index, index + 1
            del indices[begin:end]
            begin = indices[-1] + 1
            end = begin + nimages - len(indices)
            indices.extend(range(begin, end))
            cv2.imshow(window_name, fn())
            initial_index.clear()
    cv2.setMouseCallback(window_name, handle_mouse_event)
    show_and_wait(fn())
    return indices
#fn('io', selected_extents['io'][1])
cv2.destroyAllWindows()

In [25]:
# Select 1200 of each letter.
# Manually select the minimum number of zeros to include images.
nimages = 1200
ncolumns = 60
window_name = 'tesst'
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
def fn(file_title, horizontal_extent):
    images = data[file_title]
    minimum_nzero = (images[0].shape[0] - 7) * -op.sub(*selected_extents[file_title][1]) // 2
    left, right = horizontal_extent
    def fn():
        l = random.sample(images, len(images))
        g = (cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)[1] for image in l)
        g = (image[4:-3, left:right] for image in g)
        return [image for image in g if (image == 0).sum() >= minimum_nzero]
    while True:
        l = fn()
        if len(l) < nimages:
            image = np.zeros([1, 1], dtype=images[0].dtype)
        else:
            image = np.vstack([np.hstack(list(g)) for g in grouper(l[:nimages], ncolumns)])
        ch = show_and_wait(image)
        if ch == '-':
            minimum_nzero -= 1
        elif ch == '+':
            minimum_nzero += 1
        elif ch == '/':
            minimum_nzero -= 10
        elif ch == '*':
            minimum_nzero += 10
        elif ch == 'p':
            print(minimum_nzero, len(l))
        elif ch == 'q':
            return minimum_nzero, len(fn())
#print(fn('io', selected_extents['io'][1]))
cv2.destroyAllWindows()